In [5]:
import h5py
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import (LinearRegression,
                                  RidgeCV,
                                  LassoCV,
                                  ElasticNetCV)
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from uoineuro.tuning_utils import create_strf_design

%matplotlib inline

In [2]:
data = h5py.File('/storage/data/dmr/dmr_strf_data.h5', 'r')

In [3]:
n_electrodes = len(data)
stim = data['Wave1/stim'][:]
n_samples, n_features = stim.shape
resp = np.zeros((n_electrodes, n_samples))

n_frames = 20
train_frac = 0.9

In [4]:
for electrode in range(n_electrodes):
    group_name = 'Wave' + str(electrode + 1)
    group = data[group_name]
    resp[electrode] = np.squeeze(group['resp'][:])

In [7]:
X, Y = create_strf_design(stim, resp, n_frames)

In [17]:
strfs = np.zeros((n_electrodes, n_frames, n_features))
train_scores = np.zeros(n_electrodes)
test_scores = np.zeros(n_electrodes)

In [ ]:
for electrode in range(n_electrodes):
    print(electrode)
    X_train, X_test, y_train, y_test = \
        train_test_split(X, Y[electrode], train_size=train_frac)

    centering = X_train.mean(axis=0, keepdims=True)
    X_train -= centering
    X_test -= centering

    ridge = RidgeCV(alphas=np.logspace(3, 6, num=300),
                    normalize=False,
                    fit_intercept=False,
                    cv=None).fit(X_train, y_train)
    print(ridge.alpha_)
    strfs[electrode] = strfs.coef_.reshape((20, 96))
    train_scores[electrode] = ridge.score(X_train, y_train)
    test_scores[electrode] = ridge.score(X_test, y_test)
    print('---')